In [1]:
#!pip install sklearn_pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn_pandas import CategoricalImputer
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('C:/Users/shashi.singh/Downloads/train_AUpWtIz/train.csv')
cust_demograph = pd.read_csv('C:/Users/shashi.singh/Downloads/train_AUpWtIz/customer_demographics.csv')
cust_trans = pd.read_csv('C:/Users/shashi.singh/Downloads/train_AUpWtIz/customer_transaction_data.csv')
coupon_item = pd.read_csv('C:/Users/shashi.singh/Downloads/train_AUpWtIz/coupon_item_mapping.csv')
item_data = pd.read_csv('C:/Users/shashi.singh/Downloads/train_AUpWtIz/item_data.csv')
camp_data = pd.read_csv('C:/Users/shashi.singh/Downloads/train_AUpWtIz/campaign_data.csv')
test = pd.read_csv('C:/Users/shashi.singh/Downloads/test_QyjYwdj.csv')

In [3]:
cust_demograph.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,Married,0,2,NaN,4
1,6,46-55,Married,0,2,NaN,5
2,7,26-35,NaN,0,3,1,3
3,8,26-35,NaN,0,4,2,6
4,10,46-55,Single,0,1,NaN,5


In [4]:
cust_demograph['marital_status'][cust_demograph['family_size']=='1'] = cust_demograph['marital_status'][cust_demograph['family_size']=='1'].fillna('Single')
cust_demograph['marital_status'][cust_demograph['family_size']!='1'] = cust_demograph['marital_status'][cust_demograph['family_size']!='1'].fillna('Married')
cust_demograph['no_of_children'][cust_demograph['family_size']== '1'] = 0
cust_demograph['no_of_children'][cust_demograph['family_size']== '2'] = 0

In [5]:
cust_trans['item_price'] = (cust_trans['selling_price']-cust_trans['other_discount']-cust_trans['coupon_discount'])/cust_trans['quantity']
item_price = cust_trans[['item_id','item_price']].groupby('item_id').mean().reset_index()

In [6]:
coupon_item = pd.merge(coupon_item,item_data,how='inner',on = 'item_id')
coupon_item = pd.merge(coupon_item,item_price,how='inner',on = 'item_id')
coupon_item.head()

,coupon_id,item_id,brand,brand_type,category,item_price
0,105,37,56,Local,Grocery,57.247381
1,6,37,56,Local,Grocery,57.247381
2,22,37,56,Local,Grocery,57.247381
3,31,37,56,Local,Grocery,57.247381
4,107,75,56,Local,Grocery,38.828485


In [7]:
#coupon_data = pd.pivot_table(coupon_item, values='item_id', index=['coupon_id'],columns=['category'], aggfunc=lambda x: len(x.unique()))
#coupon_data = coupon_data.fillna(0)
coupon_item['category'] = pd.Series(coupon_item['category'].factorize()[0]).replace(-1, np.nan)
coupon_item['brand_type'] = pd.Series(coupon_item['brand_type'].factorize()[0]).replace(-1, np.nan)
coupon_item['brand'] = pd.Series(coupon_item['brand'].factorize()[0]).replace(-1, np.nan)
cat_agg=['nunique']
num_agg=['min','mean','max']
agg_col={
    'item_id':cat_agg, 'brand':num_agg, 'item_price':num_agg, 'brand_type':num_agg, 'category':num_agg
}
coupon_data = coupon_item.groupby('coupon_id').agg(agg_col)

In [8]:
train = train.merge(cust_demograph,how='left',on = 'customer_id')
train = train.merge(coupon_data.reset_index(),how='left',on ='coupon_id')

In [9]:
camp_data['start_date'] = pd.to_datetime(camp_data['start_date'],format='%d/%m/%y')
camp_data['end_date'] = pd.to_datetime(camp_data['end_date'],format='%d/%m/%y')
cust_trans['date'] = pd.to_datetime(cust_trans['date'],format='%Y-%m-%d')

camp_data['diff_d']=(camp_data['end_date']-camp_data['start_date'])/np.timedelta64(1,'D')
camp_data['diff_m']=(camp_data['end_date']-camp_data['start_date'])/np.timedelta64(1,'M')
camp_data['diff_w']=(camp_data['end_date']-camp_data['start_date'])/np.timedelta64(1,'W')

In [10]:
test = test.merge(cust_demograph,how='left',on = 'customer_id')
test = test.merge(coupon_data.reset_index(),how='left',on ='coupon_id')

In [11]:
train = train.fillna(train.mode().iloc[0])
test = test.fillna(train.mode().iloc[0])

In [12]:
#camp_data['diff_d']

In [13]:
i = 0
agg_col={
    'item_price':num_agg, 'item_id':cat_agg
}
_ = cust_trans[cust_trans['date'] < camp_data['start_date'][i]]
trans = _.groupby('customer_id').agg(agg_col)
trans['coupon_redeem'] = _[_['coupon_discount']< 0].groupby('customer_id').count().iloc[:,5]
trans['campaign_id'],trans['campaign_type'],trans['diff_d'],trans['diff_m'],trans['diff_w'] = camp_data['campaign_id'][i],camp_data['campaign_type'][i],camp_data['diff_d'][i],camp_data['diff_m'][i],camp_data['diff_w'][i]
#trans_feat = pd.concat([trans_feat,trans.reset_index()])

In [14]:
trans_feat = pd.DataFrame(columns=trans.reset_index().columns)
for i in range(0,len(camp_data)):
    agg_col={
        'item_price':num_agg, 'item_id':cat_agg
    }
    _ = cust_trans[cust_trans['date'] < camp_data['start_date'][i]]
    trans = _.groupby('customer_id').agg(agg_col)
    trans['coupon_redeem'] = _[_['coupon_discount']< 0].groupby('customer_id').count().iloc[:,5]
    trans['campaign_id'],trans['campaign_type'],trans['diff_d'],trans['diff_m'],trans['diff_w'] = camp_data['campaign_id'][i],camp_data['campaign_type'][i],camp_data['diff_d'][i],camp_data['diff_m'][i],camp_data['diff_w'][i]
    trans_feat = pd.concat([trans_feat,trans.reset_index()])
trans_feat['coupon_redeem'] = trans_feat['coupon_redeem'].fillna(0)
trans_feat['campaign_type'] = trans_feat['campaign_type'].astype('category')

In [15]:
train = train.merge(trans_feat, on = ['campaign_id','customer_id'],how = 'left')
test = test.merge(trans_feat, on = ['campaign_id','customer_id'],how = 'left')

In [16]:
train.isnull().sum()#.iloc[:,11:29]#.dtypes

id                          0
campaign_id                 0
coupon_id                   0
customer_id                 0
redemption_status           0
age_range                   0
marital_status              0
rented                      0
family_size                 0
no_of_children              0
income_bracket              0
('item_id', 'nunique')_x    0
(brand, min)                0
(brand, mean)               0
(brand, max)                0
('item_price', 'min')_x     0
('item_price', 'mean')_x    0
('item_price', 'max')_x     0
(brand_type, min)           0
(brand_type, mean)          0
(brand_type, max)           0
(category, min)             0
(category, mean)            0
(category, max)             0
(item_price_y, min)         0
(item_price_y, mean)        0
(item_price_y, max)         0
(item_id_y, nunique)        0
(coupon_redeem, )           0
(campaign_type, )           0
(diff_d, )                  0
(diff_m, )                  0
(diff_w, )                  0
dtype: int

In [17]:
train['rented'] = train['rented'].astype('int64')
train['campaign_type',''] = train['campaign_type',''].astype('category')
for col in ['age_range', 'marital_status', 'rented', 'family_size','no_of_children','income_bracket']:
    train[col] = train[col].astype('category')
    
train.iloc[:,26:29] = train.iloc[:,26:29].astype('float64')
#train.iloc[:,27] = train.iloc[:,27].astype('int64')

train1 = train.iloc[:,11:29]

test['rented'] = test['rented'].astype('int64')
test['campaign_type',''] = test['campaign_type',''].astype('category')
for col in ['age_range', 'marital_status', 'rented', 'family_size','no_of_children','income_bracket']:
    test[col] = test[col].astype('category')
    
#test.iloc[:,10:43] = test.iloc[:,10:43].astype('int64')
test.iloc[:,10:27] = test.iloc[:,10:27].astype('int64')
test1 = test.iloc[:,10:28]

In [27]:
train.dtypes

id                             int64
campaign_id                   object
coupon_id                      int64
customer_id                   object
redemption_status              int64
age_range                   category
marital_status              category
rented                      category
family_size                 category
no_of_children              category
income_bracket              category
('item_id', 'nunique')_x       int64
(brand, min)                   int64
(brand, mean)                float64
(brand, max)                   int64
('item_price', 'min')_x      float64
('item_price', 'mean')_x     float64
('item_price', 'max')_x      float64
(brand_type, min)              int64
(brand_type, mean)           float64
(brand_type, max)              int64
(category, min)                int64
(category, mean)             float64
(category, max)                int64
(item_price_y, min)          float64
(item_price_y, mean)         float64
(item_price_y, max)          float64
(

In [18]:
X,y=train1,train['redemption_status']
Xtest=test1

In [19]:
train['redemption_status'].value_counts()/train.shape[0]

0    0.990698
1    0.009302
Name: redemption_status, dtype: float64

In [20]:

from catboost import CatBoostClassifier,Pool, cv
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score

In [21]:
err=[]
y_pred_tot=[]
from sklearn.model_selection import KFold,StratifiedKFold
fold=StratifiedKFold(n_splits=20,shuffle=True,random_state=1994)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m=LGBMClassifier(n_estimators=3000,random_state=1994,learning_rate=0.02,colsample_bytree=0.2,objective='binary',scale_pos_weight=1)
    m.fit(X_train,y_train,eval_set=[(X_test, y_test)],eval_metric='auc', early_stopping_rounds=200,verbose=200)
    preds=m.predict_proba(X_test)[:,-1]
    print("err: ",roc_auc_score(y_test,preds))
    err.append(roc_auc_score(y_test,preds))
    p = m.predict_proba(Xtest)[:,-1]
    i=i+1
    y_pred_tot.append(p)

Training until validation scores don't improve for 200 rounds
[200]	valid_0's auc: 0.929989	valid_0's binary_logloss: 0.0385537
[400]	valid_0's auc: 0.937376	valid_0's binary_logloss: 0.0369938
[600]	valid_0's auc: 0.938315	valid_0's binary_logloss: 0.0362443
[800]	valid_0's auc: 0.937689	valid_0's binary_logloss: 0.0360324
Early stopping, best iteration is:
[693]	valid_0's auc: 0.938977	valid_0's binary_logloss: 0.0360798
err:  0.9389768439227482
Training until validation scores don't improve for 200 rounds
[200]	valid_0's auc: 0.895199	valid_0's binary_logloss: 0.039932
Early stopping, best iteration is:
[179]	valid_0's auc: 0.900239	valid_0's binary_logloss: 0.0399276
err:  0.9002394976119861
Training until validation scores don't improve for 200 rounds
[200]	valid_0's auc: 0.883238	valid_0's binary_logloss: 0.0379792
[400]	valid_0's auc: 0.892136	valid_0's binary_logloss: 0.0366588
[600]	valid_0's auc: 0.894886	valid_0's binary_logloss: 0.0363478
[800]	valid_0's auc: 0.893855	valid

err:  0.9300689793348216
Training until validation scores don't improve for 200 rounds
[200]	valid_0's auc: 0.874571	valid_0's binary_logloss: 0.0406834
[400]	valid_0's auc: 0.890141	valid_0's binary_logloss: 0.0392657
[600]	valid_0's auc: 0.89847	valid_0's binary_logloss: 0.0387866
Early stopping, best iteration is:
[567]	valid_0's auc: 0.89857	valid_0's binary_logloss: 0.0387104
err:  0.8985703245749613
Training until validation scores don't improve for 200 rounds
[200]	valid_0's auc: 0.927915	valid_0's binary_logloss: 0.0366931
[400]	valid_0's auc: 0.941597	valid_0's binary_logloss: 0.034314
[600]	valid_0's auc: 0.942126	valid_0's binary_logloss: 0.0338193
[800]	valid_0's auc: 0.945246	valid_0's binary_logloss: 0.0331412
[1000]	valid_0's auc: 0.945189	valid_0's binary_logloss: 0.0329862
Early stopping, best iteration is:
[841]	valid_0's auc: 0.945818	valid_0's binary_logloss: 0.0329574
err:  0.9458183067147519


In [22]:
np.mean(err,0)

0.9248874550748587

In [23]:
sub = pd.read_csv('C:/Users/shashi.singh/Downloads/sample_submission_Byiv0dS.csv')

In [25]:
sub['redemption_status']=np.mean(y_pred_tot,0)
sub.head()

,id,redemption_status
0,3,0.084806
1,4,0.002192
2,5,0.030835
3,8,0.001510
4,10,0.001742


In [28]:
sub.to_csv('C:/Users/shashi.singh/Downloads/sample_6th_nocat_pivot.csv',index=False)
sub.shape

(50226, 2)

In [29]:
X,y=train.drop(['id','customer_id','campaign_id','coupon_id','redemption_status'],axis=1),train['redemption_status']
Xtest=test.drop(['id','customer_id','campaign_id','coupon_id'],axis=1)

print(X.shape,Xtest.shape)
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.25,random_state = 1994,stratify=y)
categorical_features_indices = np.where(X_train.dtypes =='category')[0]
categorical_features_indices

(78369, 28) (50226, 28)


array([ 0,  1,  2,  3,  4,  5, 24], dtype=int64)

In [30]:
m=CatBoostClassifier(n_estimators=3000,random_state=1994,learning_rate=0.03,eval_metric='AUC')
m.fit(X_train,y_train,eval_set=[(X_val, y_val.values)], early_stopping_rounds=200,verbose=200,cat_features=categorical_features_indices)
p=m.predict_proba(X_val)[:,-1]
print(roc_auc_score(y_val,p))

0:	test: 0.6915768	best: 0.6915768 (0)	total: 211ms	remaining: 10m 33s
200:	test: 0.8832377	best: 0.8832377 (200)	total: 22s	remaining: 5m 5s
400:	test: 0.8952296	best: 0.8952709 (397)	total: 45.9s	remaining: 4m 57s
600:	test: 0.9015906	best: 0.9015906 (600)	total: 1m 9s	remaining: 4m 38s
800:	test: 0.9061849	best: 0.9061849 (800)	total: 1m 34s	remaining: 4m 19s
1000:	test: 0.9086360	best: 0.9087014 (999)	total: 1m 58s	remaining: 3m 56s
1200:	test: 0.9113330	best: 0.9114479 (1193)	total: 2m 23s	remaining: 3m 34s
1400:	test: 0.9131760	best: 0.9131873 (1399)	total: 2m 46s	remaining: 3m 10s
1600:	test: 0.9140566	best: 0.9141175 (1593)	total: 3m 10s	remaining: 2m 46s
1800:	test: 0.9152347	best: 0.9153660 (1795)	total: 3m 34s	remaining: 2m 23s
2000:	test: 0.9159676	best: 0.9159676 (2000)	total: 3m 59s	remaining: 1m 59s
2200:	test: 0.9154453	best: 0.9160380 (2023)	total: 4m 23s	remaining: 1m 35s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.9160380344
bestIteration = 2

In [31]:
errCB=[]
y_pred_tot_cb=[]
from sklearn.model_selection import KFold,StratifiedKFold
fold=StratifiedKFold(n_splits=20,shuffle=True,random_state=1994)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m=CatBoostClassifier(n_estimators=2500,random_state=1994,eval_metric='AUC',learning_rate=0.05)
    m.fit(X_train,y_train,eval_set=[(X_test, y_test)], early_stopping_rounds=200,verbose=200,cat_features=categorical_features_indices)
    preds=m.predict_proba(X_test)[:,-1]
    print("err_cb: ",roc_auc_score(y_test,preds))
    errCB.append(roc_auc_score(y_test,preds))
    p = m.predict_proba(Xtest)[:,-1]
    i=i+1
    y_pred_tot_cb.append(p)

0:	test: 0.6258372	best: 0.6258372 (0)	total: 141ms	remaining: 5m 53s
200:	test: 0.9155910	best: 0.9155910 (200)	total: 25.8s	remaining: 4m 54s
400:	test: 0.9236880	best: 0.9236880 (400)	total: 51.4s	remaining: 4m 28s
600:	test: 0.9308660	best: 0.9308660 (600)	total: 1m 17s	remaining: 4m 3s
800:	test: 0.9367350	best: 0.9369787 (795)	total: 1m 42s	remaining: 3m 38s
1000:	test: 0.9380927	best: 0.9387053 (925)	total: 2m 8s	remaining: 3m 12s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.9387053205
bestIteration = 925

Shrink model to first 926 iterations.
err_cb:  0.9387053204672989
0:	test: 0.6511829	best: 0.6511829 (0)	total: 135ms	remaining: 5m 36s
200:	test: 0.8898520	best: 0.8904925 (192)	total: 25.3s	remaining: 4m 49s
400:	test: 0.9005249	best: 0.9005806 (399)	total: 50.9s	remaining: 4m 26s
600:	test: 0.9000515	best: 0.9010471 (540)	total: 1m 16s	remaining: 4m 2s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.9010471058
bestIteration = 540

1200:	test: 0.9175969	best: 0.9176971 (1015)	total: 2m 40s	remaining: 2m 53s
1400:	test: 0.9177758	best: 0.9192784 (1285)	total: 3m 7s	remaining: 2m 27s
1600:	test: 0.9242515	best: 0.9242515 (1600)	total: 3m 35s	remaining: 2m
1800:	test: 0.9252891	best: 0.9252891 (1800)	total: 4m 3s	remaining: 1m 34s
2000:	test: 0.9247954	best: 0.9261692 (1934)	total: 4m 31s	remaining: 1m 7s
2200:	test: 0.9278293	best: 0.9279438 (2192)	total: 4m 59s	remaining: 40.6s
2400:	test: 0.9279867	best: 0.9292103 (2338)	total: 5m 27s	remaining: 13.5s
2499:	test: 0.9284661	best: 0.9292103 (2338)	total: 5m 41s	remaining: 0us

bestTest = 0.9292103154
bestIteration = 2338

Shrink model to first 2339 iterations.
err_cb:  0.9292103154158795
0:	test: 0.7395493	best: 0.7395493 (0)	total: 134ms	remaining: 5m 34s
200:	test: 0.8811752	best: 0.8818979 (196)	total: 26s	remaining: 4m 57s
400:	test: 0.8886456	best: 0.8887672 (397)	total: 51.2s	remaining: 4m 28s
600:	test: 0.9005023	best: 0.9005023 (600)	total: 1m 17s	remaining

In [32]:
np.mean(errCB,0)

0.9178297598216453

In [256]:
sub = pd.read_csv('C:/Users/shashi.singh/Downloads/sample_submission_Byiv0dS.csv')

In [257]:
sub['redemption_status']=np.mean(y_pred_tot_cb,0)
sub.head()

,id,redemption_status_x,campaign_id,coupon_id,customer_id,age_range,marital_status,rented,family_size,no_of_children,...,"(category, min)","(category, mean)","(category, max)","(item_price_y, min)","(item_price_y, mean)","(item_price_y, max)","(item_id_y, nunique)","(coupon_redeem, )","(campaign_type, )",redemption_status_y
0,3,0,22,869,967,36-45,Single,0,1,0,...,0,0,0,0,117,1276,658,81.0,X,0.223376
1,4,0,20,389,1566,26-35,Married,0,2,0,...,3,3,3,0,96,1780,1214,31.0,Y,0.003504
2,5,0,22,981,510,26-35,Married,0,2,0,...,0,0,0,0,86,1673,1019,56.0,X,0.066947
3,8,0,25,1069,361,18-25,Single,0,1,0,...,0,0,0,0,88,1638,323,0.0,Y,0.003074
4,10,0,17,498,811,46-55,Married,0,2,0,...,0,0,0,6,85,1281,720,3.0,Y,0.001658


In [258]:
sub.to_csv('C:/Users/shashi.singh/Downloads/sample_6th_pivot.csv',index=False)
sub.shape

(50226, 31)

In [821]:
sub['redemption_status']=np.mean(y_pred_tot_cb,0)*0.65 + np.mean(y_pred_tot,0)*0.35

In [822]:
sub.to_csv('C:/Users/shashi.singh/Downloads/sample_combined_second.csv',index=False)
sub.shape

(50226, 2)